In [0]:
%run ../parse_sql

In [0]:
query_string = """
SELECT
    psatt.fund_id AS fund_id,
    psatt.cal_dt AS calen_dt,
    psa.asset_id AS asset_id,

    CASE 
        -- psa.int_rt
        WHEN rsm.investment_type = 'BL'
             AND NVL(psa.acrd_cd, 'N') NOT IN ('Y', 'V')  -- Story 897300
        THEN NVL(
            (SELECT wacoupon
             FROM stg_wso_pos_acr_ame
             WHERE portfolio_fundid = psa.fund_id
               AND asofdate = psa.cal_dt
               AND asset_primaryid = psa.asset_id
               AND rec_typ_cd = 'POS'),
            0
        )
        ELSE psa.int_rt
    END AS pos_int_rt,

    CASE
        WHEN psa.fund_asset_cls_cd IN ('OP', 'OW', 'FL', 'FS')
        THEN psa.mtrty_dt
        ELSE rsm.e_expire_dt
    END AS maturity_or_expiry_dt,

    CASE
        WHEN NVL(psah.acrd_cd, 'N') NOT IN ('Y', 'V')
        THEN NVL(
            (SELECT MAX(rateoption_floorrate)
             FROM stg_wso_con_dtl_ame
             WHERE portfolio_fundid = psah.fund_id
               AND asofdate = psah.cal_dt
               AND asset_primaryid = psah.asset_id
               AND rec_typ_cd = 'POS'),
            0
        )
        ELSE NULL
    END AS floor_base_int_rt

FROM 
    pos_sum_ame_tot psatt
JOIN pos_sum_ame_tot psatc ON psatt.fund_id = psatc.fund_id
                           AND psatt.cal_dt = psatc.cal_dt
JOIN pos_sum_ame_hist psah ON psah.fund_id = psatt.fund_id
                           AND psah.cal_dt = psatt.cal_dt
JOIN ref_fund_status rfs ON psatt.fund_id = rfs.fund_id
                         AND rfs.period_ind_cd = 'M'
JOIN dhr_t dhr ON psatt.fund_id = dhr.ssb_fund
              AND dhr.ssb_fund = rfs.fund_id
JOIN rsm rsm ON psah.security_id = rsm.rom_instance
LEFT JOIN pos_sum_ame_hist psa ON psa.fund_id = psatt.fund_id 
                               AND psa.cal_dt = psatt.cal_dt 
                               AND psa.asset_id = psah.asset_id

WHERE
    psatt.fnd_tot_typ_cd = 'T'
    AND psah.base_curr_cd = psatt.fnd_tot_curr_cd
    AND psah.local_curr_cd = psatc.fnd_tot_curr_cd
    AND psatc.fnd_tot_typ_cd = 'C'
    AND (psah.shr_par_qty < 0 OR psah.cst_mthd_lam > 0);
"""

print_full_queries(query_string, extract_columns_flag=True)

In [0]:
subquery_df = get_split_sql_as_dataframe(query_string, extract_columns_flag=True)
display(subquery_df.select("name", "original", "columns"))

In [0]:
converted_df = convert_and_get_dataframe(
    query_string, endpoint_name="databricks-claude-3-7-sonnet", extract_columns_flag=True, test_mode=True
)
display(
    converted_df.select("name", "original", "columns", "converted", "converted_columns", "response_error", "status")
)

In [0]:
assembled_df = assemble_final_query(converted_df)
display(assembled_df)

In [0]:
prettified_value = prettify_final(get_main(assembled_df))
print(prettified_value)